In [67]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mlflow as mlflow

# Hyper Parameters
batch_size = 64
block_size = 128
n_embd = 32
n_head = 4
nvocab = 65
bias = False
dropout = 0.2
Bias = False
import math

In [113]:
torch.manual_seed(1337)
c = nn.Linear(n_embd,3*n_embd)

In [117]:
c(d).shape[2] 

96

In [ ]:
torch.manual_seed(1337)
embedding_table = nn.Embedding(nvocab,n_embd)
positional_embedding = nn.Embedding(block_size,n_embd)

In [77]:
torch.manual_seed(1337)
d = embedding_table(torch.randint(65,(batch_size,block_size)))+positional_embedding(torch.arange(block_size))

In [140]:
class CasualSelfAttention1(nn.Module):
    def __init__(self):
        torch.manual_seed(1337)
        super().__init__()
        
        assert n_embd % n_head == 0
        self.headsize = n_embd//n_head
        self.csAttn = nn.Linear(n_embd,3 *n_embd ,bias = Bias)
        self.mh_proj = nn.Linear(n_embd,n_embd ,bias = Bias)
        self.register_buffer("tril",torch.tril(torch.ones(block_size,block_size)).view(1,1,block_size,block_size))
        self.attdropout = nn.Dropout(dropout)
        self.mhdropout = nn.Dropout(dropout)
    
    def forward(self,x):
        torch.manual_seed(1337)
        B,T,C = x.shape
        q,k,v = self.csAttn(x).split(n_embd,dim=-1)
        
        k = k.view(B,T,n_head,C // n_head).transpose(1,2)
        q = q.view(B,T,n_head,C // n_head).transpose(1,2)
        v = v.view(B,T,n_head,C // n_head).transpose(1,2)
        
        wei = (q @ k.transpose(-2,-1)) * (1.0/math.sqrt(k.size(-1)))
        wei = wei.masked_fill(self.tril[:,:,:T,:T]==0,float('-inf'))
        wei = F.softmax(wei,dim=-1)
        wei = self.attdropout(wei)
        out = wei @ v
        out = out.transpose(1,2).view(B,T,C)
        out = self.mhdropout(self.mh_proj(out))
        return out


In [141]:
class CausalSelfAttention(nn.Module):

    def __init__(self):
        super().__init__()
        assert n_embd % n_head == 0
        
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(n_embd, 3 * n_embd, bias=bias)
        # output projection
        self.c_proj = nn.Linear(n_embd, n_embd, bias=bias)
        # regularization
        self.attn_dropout = nn.Dropout(dropout)
        self.resid_dropout = nn.Dropout(dropout)
        self.n_head = n_head
        self.n_embd = n_embd
        self.dropout = dropout
        # flash attention make GPU go brrrrr but support is only in PyTorch >= 2.0
        print("WARNING: using slow attention. Flash Attention requires PyTorch >= 2.0")
        # causal mask to ensure that attention is only applied to the left in the input sequence
        self.register_buffer("bias", torch.tril(torch.ones(block_size, block_size))
                                    .view(1, 1, block_size, block_size))

    def forward(self, x):
        torch.manual_seed(1337)
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        q, k, v  = self.c_attn(x).split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        # print(k.shape,q.shape,v.shape)
        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)

        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        att = self.attn_dropout(att)
        y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_dropout(self.c_proj(y))
        return y


In [142]:
torch.manual_seed(1337)
ca = CausalSelfAttention()
cb = CasualSelfAttention1()

In [143]:
ca(d) == cb(d)

RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

In [97]:
d.shape[-1],d.size(-1)

(32, 32)

In [134]:
flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention')

In [135]:
flash

True